In [1]:
from datetime import datetime
from typing import List, Sequence
from config import OPENAI_API_KEY, TAVILY_API_KEY, LANGCHAIN_TRACING_V2, LANGCHAIN_API_KEY
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser, PydanticToolsParser
from langgraph.graph import END, MessageGraph
from langchain_openai import ChatOpenAI
from schemas import AnswerQuestion, ReviseAnswer

In [2]:
llm = ChatOpenAI(model="gpt-4-turbo-2024-04-09")
# it takes the output from the llm and transform it into a dictionary
parser = JsonOutputToolsParser(return_id=True)
# it takes the output from the llm and transform it into an AnswerQuestion object
parser_pydantic =  PydanticToolsParser(tools=[AnswerQuestion])

In [3]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
        """
        You are an expert researcher.
        Current time: {time}
        
        1. {first_instruction}
        2. Reflect and Critique your answer. Be severe to maximize improvements.
        3. Recommend search queries to research information and improve your answer.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
).partial(time=lambda: datetime.now().isoformat())

In [4]:
first_responder_prompt_template = actor_prompt_template.partial(first_instruction="Provide a detailed ~250 word answer.")

In [5]:
first_responder = first_responder_prompt_template | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")

In [6]:
revise_instruction = """
Revise your previous answer using the new information.
- You should use the previous critique to add important information to your answer.
- You MUST include numerical citations in your revised answer to ensure it can be verified.
- Add a "References" section to the bottom of your answer (which does not count towards the word limit) in the form of:
    - [1] https://example.com
    - [2] https://example.com
- You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
"""

In [8]:
revisor = actor_prompt_template.partial(first_instruction=revise_instruction) | \
            llm.bind_tools(tools=[ReviseAnswer], tool_choice="ReviseAnswer")

In [ ]:
human_message = HumanMessage(
    content="Write about the use of LLM as a planner in Reinforcement Learning")

In [ ]:
chain =  first_responder_prompt_template |  llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion") | parser_pydantic

In [ ]:
res = chain.invoke(input={"messages": [human_message]})[0]

In [ ]:
print(res)

In [ ]:
res.answer

In [ ]:
res.reflection

In [ ]:
res.search_queries